In [29]:
from spire.doc import *
from spire.doc.common import *
import json
import re

In [30]:
# Create a Document object
document = Document()

In [31]:
def extract_text():

    # Load a Word document
  document.LoadFromFile("CoursePlans/Syllabus_MCA272– Programming Using Java.docx")

  # Extract the text of the document
  document_text = document.GetText()

  #Substitutes '\t' with 1 '\n'
  document_text = re.sub(r'\t{1,}', '\n', document_text)
  #Trim the spaces after newline
  document_text = re.sub(r'\n\s+', '\n', document_text)
  #Substitutes '\r' with ''
  document_text = document_text.replace('\r', '')

  with open("Output/DocumentText.txt", "w", encoding="utf-8") as file:
    file.write(document_text)

In [32]:
# JSON Structure:

# [
#   CourseName-Num: ,
#   Total-Teaching-Hours: ,
#   Max-Marks: ,
#   Credits: ,
#   Course-Outcomes: [
#     CO1: ,
#   CO2: ,
#   CO3: ,
#   ],
#   Unit1: [
#     Teaching-hours: ,
#     Title: ,
#     Contents: [],
#     Lab-exercises: [],
#         ],
#   Unit2: [
#     Teaching-hours: ,
#     Title: ,
#     Contents: [],
#     Lab-exercises: [],
#         ],
#   Unit3: [
#     Teaching-hours: ,
#     Title: ,
#     Contents: [],
#     Lab-exercises: [],
#         ],
#   Unit4: [
#     Teaching-hours: ,
#     Title: ,
#     Contents: [],
#     Lab-exercises: [],
#         ],
#   Unit5: [
#     Teaching-hours: ,
#     Title: ,
#     Contents: [],
#     Lab-exercises: [],
#         ],
#   Text-books: [],
#   Web-resources: [],
# ]

In [33]:
def storeJson(input_file, output_file):
    # Read the content of the input file
    with open(input_file, 'r') as file:
        processed_file = file.read()

    # Initialize storage for extracted data
    extracted_data = {
        "CourseName-Num": "",
        "Total-Teaching-Hours": "",
        "Max-Marks": "",
        "Credits": "",
        "Course-Outcomes": [],
        "Units": [],
        "Text-books": [],
        "Web-resources": []
    }

    # Extract basic course details
    extracted_data["CourseName-Num"] = re.search(r'^MCA\d{3}– [^\n]+', processed_file, re.MULTILINE).group(0).strip()
    extracted_data["Total-Teaching-Hours"] = re.search(r'Total Teaching Hours for Semester: (\d+)', processed_file).group(1)
    extracted_data["Max-Marks"] = re.search(r'Max Marks: (\d+)', processed_file).group(1)
    extracted_data["Credits"] = re.search(r'Credits: (\d+)', processed_file).group(1)

    # Extract course outcomes
    outcomes = re.findall(r'CO\d+: (.+)', processed_file)
    for idx, outcome in enumerate(outcomes, start=1):
        extracted_data["Course-Outcomes"].append({f"CO{idx}": outcome.strip()})

    # Extract unit details
    unit_pattern = r'Unit-(\d+)\nTeaching Hours: (\d+)\n([^\n]+)\n(.+?)(?=Unit-\d+|Text Books and Reference Books)'
    units = re.findall(unit_pattern, processed_file, re.DOTALL)
    for unit in units:
        unit_data = {
            "Teaching-hours": unit[1],
            "Title": unit[2].strip(),
            "Contents": [content.strip() for content in re.split(r'(?<=\.)\s+', unit[3].strip()) if content],
            "Lab-exercises": re.findall(r'Lab Exercises:\n(.+)', unit[3], re.DOTALL)
        }
        extracted_data["Units"].append(unit_data)

    # Extract text-books
    textbooks = re.findall(r'\[\d+\] ([^\n]+)', processed_file)
    extracted_data["Text-books"] = textbooks

    # Extract web resources
    web_resources = re.findall(r'https?://[^\s]+', processed_file)
    extracted_data["Web-resources"] = web_resources

    # Save extracted data to a JSON file
    with open(output_file, 'w') as json_file:
        json.dump(extracted_data, json_file, indent=4)

In [34]:
input_file = "Output/DocumentText.txt"
output_file = "Output/CourseDetails.json"

# Write the extracted text into a text file
extract_text()
storeJson(input_file, output_file)


In [35]:
document.Close()

# Things needed (external)
- Evaluation Pattern (CIA and ETE)
- Trimester
- Faculty name and contacts
- Class policies and description
- CIA and ETE component description (like Practical tests, Group project, mcqs, presentations, etc)
- Duration of the course (Start and End Date)